<a href="https://colab.research.google.com/github/tempcheng/python_homework/blob/master/hotel_recco_hw_emily.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving booking_com-travel_sample.csv to booking_com-travel_sample.csv


In [ ]:
import io
hotels = pd.read_csv(io.BytesIO(uploaded['booking_com-travel_sample.csv']))

In [ ]:
hotels.head()

,address,city,country,crawl_date,hotel_brand,hotel_description,hotel_facilities,hotel_star_rating,image_count,latitude,locality,longitude,pageurl,property_id,property_name,property_type,province,qts,room_count,room_type,similar_hotel,site_review_count,site_review_rating,site_stay_review_rating,sitename,special_tag,state,uniq_id,zone
0,"KHIRSU, 246147 Pauri, India – Great location -",pauri,India,2016-09-01,NaN,Khirsu By GMVN offers accommodation in Pauri. ...,Bathroom:Toilet paper|Linen|Towels|Bathroom|To...,NaN,3.0,30.123749,NaN,78.793774,http://www.booking.com/hotel/in/khirsu-by-gmvn...,1742658.0,Khirsu By GMVN,204,NaN,2016-09-01 11:52:38 +0000,18.0,Economy Double Room,Hotel Mandakini,NaN,NaN,NaN,http://www.booking.com/,Share,Uttarakhand,a5ea72415f8007fcbe65759830fdd3d9,NaN
1,"Kaathadimattam, Balacola Post, NEAR Siva Tea F...",ooty,India,2016-09-01,NaN,"Situated in Ooty in the Tamil Nadu Region, 8 k...",Bathroom:Toilet paper|Linen|Towels|Bidet|Towel...,3 stars,NaN,11.329595,NaN,76.686732,http://www.booking.com/hotel/in/global-village...,1755986.0,Global Village Ooty,212,NaN,2016-09-01 11:52:38 +0000,10.0,British Empire Chalet,Treebo Yantra Leisures|Western Valley Resorts|...,5,7.6,Location:8.5|Staff:8|Cleanliness:7.5|Comfort:7...,http://www.booking.com/,Share,Tamil Nadu,7e0b055417271bbd9dae902f3e231ed4,NaN
2,"PIPALKOTI, 246472 Pīpalkoti, India – Show map",pīpalkoti,India,2016-09-01,NaN,TRH Pipalkoti offers accommodation in Pīpalkot...,Bathroom:Toilet paper|Linen|Towels|Bathroom•Vi...,NaN,4.0,30.429540,NaN,79.428116,http://www.booking.com/hotel/in/trh-pipalkoti....,1741747.0,TRH Pipalkoti,204,NaN,2016-09-01 11:52:38 +0000,28.0,Economy Double Room,TRH Joshimath (New),NaN,NaN,NaN,http://www.booking.com/,Share,Uttarakhand,72c0af09827bbb620365aa5df523ba1d,NaN
3,"1 KARIYIL HOUSE KUMARAKOM NORTH PO KOTTAYAM, 6...",kumarakom,India,2016-09-01,NaN,"Swasti house boat 2 is located in Kumarakom, 3...",Bathroom:Toilet paper|Towels|Bath|Shower•Bedro...,NaN,2.0,9.616057,NaN,76.428269,http://www.booking.com/hotel/in/swasti-house-b...,1747392.0,Swasti house boat 2,204,NaN,2016-09-01 11:52:38 +0000,1.0,Deluxe Room,NaN,NaN,NaN,NaN,http://www.booking.com/,Share,Kerala,eb6fd33d99aa4a8088caa8f3ecb08275,NaN
4,"Kavanattinkara, 686563 Kumarakom, India – Show...",kumarakom,India,2016-09-01,NaN,"Amrutham Houseboat 2 is set in Kumarakom, 5 km...",Bathroom:Toilet paper|Linen|Towels|Towels/Shee...,NaN,NaN,9.632854,NaN,76.423570,http://www.booking.com/hotel/in/amrutham-house...,1744962.0,Amrutham Houseboat 2,215,NaN,2016-09-01 11:52:38 +0000,2.0,Mobile Home,Mandala Beach House & Cottages,NaN,NaN,NaN,http://www.booking.com/,Share,Kerala,0814d9af7ad808863c04db8f30437c57,NaN


In [ ]:
hotels['hotel_description'].head()

0    Khirsu By GMVN offers accommodation in Pauri. ...
1    Situated in Ooty in the Tamil Nadu Region, 8 k...
2    TRH Pipalkoti offers accommodation in Pīpalkot...
3    Swasti house boat 2 is located in Kumarakom, 3...
4    Amrutham Houseboat 2 is set in Kumarakom, 5 km...
Name: hotel_description, dtype: object

In [ ]:
#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
hotels['hotel_description'] = hotels['hotel_description'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(hotels['hotel_description'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(6000, 5249)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [ ]:
cosine_sim.shape

(6000, 6000)

In [ ]:
indices = pd.Series(hotels.index, index=hotels['property_name']).drop_duplicates()

In [ ]:
indices[:10]

property_name
Khirsu By GMVN          0
Global Village Ooty     1
TRH Pipalkoti           2
Swasti house boat 2     3
Amrutham Houseboat 2    4
The Hill Side           5
Hotel Mountain Face     6
Rahi Motel Haridwar     7
Swasti house boat 4     8
NaN                     9
dtype: int64

In [ ]:
def get_recommendations(property_name, cosine_sim=cosine_sim):
    # Get the index of the property that matches the title
    idx = indices[property_name]

    if isinstance(idx, pd.core.series.Series):
       idx = idx[0]

    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:20]

    hotel_indices = [i[0] for i in sim_scores]

    return hotels['property_name'].iloc[hotel_indices]

In [ ]:
get_recommendations('Khirsu By GMVN')

4585                   Hotel Grand Plaza
13                           Grand Hotel
3047                   Hotel N.M.R.Plaza
3051                   Hotel N.M.R.Plaza
51                 Hotel Royal Residency
175                Hotel Royal Residency
199                         Ekant Resort
4112    Vista Rooms near Railway Station
4337    Vista Rooms near Railway Station
276                    Home In The Hills
315                    Home In The Hills
327                    Home In The Hills
7                    Rahi Motel Haridwar
5616                   Ark Inn Residency
5686                   Ark Inn Residency
5811                   Ark Inn Residency
5866                   Ark Inn Residency
4151              Blue Diamond The Hotel
4157              Blue Diamond The Hotel
Name: property_name, dtype: object

In [ ]:
A = hotels.loc[hotels['property_name'] == 'Home In The Hills']



In [ ]:
A['hotel_description']

276    Featuring free WiFi and a children's playgroun...
315    Featuring free WiFi and a children's playgroun...
327    Featuring free WiFi and a children's playgroun...
Name: hotel_description, dtype: object